In [25]:
import pandas as pd
import scipy.io as sio
from scipy import signal, stats
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

riverDF = pd.read_csv('river_temperature13_14.csv', parse_dates = ['DN'])
riverDF['hourDN'] = riverDF['DN'].dt.floor('h')

hourlyMeanRiverDF = riverDF.groupby('hourDN').agg(hourlyTemp=('EstimatedTemp', "mean")).reset_index()
hourlyMeanRiverDF['hourSalinity'] = 0

heatFluxModelDF = pd.read_csv('river_2013-2014_flux.csv', parse_dates = ['Tm (UTC)'])
heatFluxModelDF = pd.merge(left = hourlyMeanRiverDF, right = heatFluxModelDF, how = 'inner',  left_on = 'hourDN', right_on = 'Tm (UTC)')
heatFluxModelDF = heatFluxModelDF.drop('Tm (UTC)', axis = 1)
jdDateBase = pd.Timestamp('1858-11-17').to_julian_date()
heatFluxModelDF['jdTimeStamp'] = heatFluxModelDF['hourDN'].apply(lambda x: x.to_julian_date()- jdDateBase)

pd.set_option('display.float_format', '{:.20f}'.format)

print(heatFluxModelDF['jdTimeStamp']+ jdDateBase)

0      2456596.12500000000000000000
1      2456596.16666666651144623756
2      2456596.20833333348855376244
3      2456596.25000000000000000000
4      2456596.29166666651144623756
                   ...             
3190   2456729.04166666651144623756
3191   2456729.08333333348855376244
3192   2456729.12500000000000000000
3193   2456729.16666666651144623756
3194   2456729.20833333348855376244
Name: jdTimeStamp, Length: 3195, dtype: float64
